In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from PIL import ImageDraw
import os
import detect
#import tflite as tf
import platform
import datetime
import cv2
import time
import numpy as np
import io
from io import BytesIO
from flask import Flask, request, Response, jsonify
import random
import re
import base64

In [2]:
def display_image(image):
  fig = plt.figure(figsize=(20, 15))
  plt.grid(False)
  plt.imshow(image)

In [3]:
folder_path = r'C:\Users\maxsc\Desktop\TU\DIC3\Pictures_small'
images_s = []

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    # Construct the full file path
    file_path = os.path.join(folder_path, filename)

    # Check if the file is a JPEG image
    if os.path.isfile(file_path) and filename.lower().endswith(".jpg"):
        with open(file_path, "rb") as file:
            # Read the image data
            image_data = file.read()

            # Encode the image data as base64
            encoded_data = base64.b64encode(image_data).decode("utf-8")

            # Append the encoded string to the list
            images_s.append(encoded_data)

print(len(images_s))


296


In [6]:
#Preprocess ONE image


#example = images_s[32]

#image_bytes = base64.b64decode(example)
#image = Image.open(io.BytesIO(image_bytes))
#image = image.resize((300, 300))  # Resize to match the model's input size
#image_np = np.array(image).astype(np.float32)  # Convert PIL image to NumPy array

#image_np_expanded = np.expand_dims(image_np, axis=0)
#image_np = image_np.astype(np.float32)
#image_np = np.expand_dims(image_np, axis=0)
imgs = []

for i in images_s:
    image_bytes = base64.b64decode(i)
    image = Image.open(io.BytesIO(image_bytes))
    image = image.resize((300, 300))  # Resize to match the model's input size
    image_np = np.array(image).astype(np.float32)  # Convert PIL image to NumPy array
    imgs.append(image_np)


len(imgs)

296

In [4]:
#Load detector model

model_path = r"https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"
detector = hub.load(model_path).signatures['default']


#IGNORE THIS PART
#interpreter = tf.lite.Interpreter(model_path=model_path)
#interpreter.allocate_tensors()

#Example image
#input_tensor = interpreter.get_input_details()[0]['index']
#interpreter.set_tensor(input_tensor, image_np)

#interpreter.invoke()





INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [7]:
#CODE TO DO OBJECT DETECTION ON ONE IMAGE
img_nr = 4


converted_img  = tf.image.convert_image_dtype(imgs[img_nr], tf.float32)[tf.newaxis, ...]

start_time = time.time()
result = detector(converted_img)
end_time = time.time()
result = {key:value.numpy() for key,value in result.items()}

print("Found %d objects." % len(result["detection_scores"]))
print("Inference time: ", end_time-start_time)

Found 0 objects.
Inference time:  81.36582040786743


In [ ]:
#NOT RELEVANT RIGHT NOW
# Get the output tensors
output_details = interpreter.get_output_details()
boxes = interpreter.get_tensor(output_details[0]['index'])
classes = interpreter.get_tensor(output_details[1]['index'])
scores = interpreter.get_tensor(output_details[2]['index'])
num_detections = int(output_details[3]['index'][0])

In [ ]:
#NOT RELEVANT RIGHT NOW
# Filter out detections with low confidence
threshold = 0.5
detections = []
for i in range(num_detections):
    if scores[0][i] > threshold:
        class_id = int(classes[0][i])
        class_name = label_map[class_id]
        box = boxes[0][i]
        detection = {
            'class': class_name,
            'score': float(scores[0][i]),
            'box': [float(box[0]), float(box[1]), float(box[2]), float(box[3])]
        }
        detections.append(detection)